In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_next_27_hour,rides_next_28_hour,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour
0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,10.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,1.0,2.0,5.0,2.0,2.0,2.0
2,0.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,5.0,4.0,1.0,8.0
3,4.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,4.0,...,2.0,1.0,1.0,0.0,1.0,1.0,7.0,2.0,2.0,3.0
4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
111212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,0.0
111213,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0
111214,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0


In [2]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    targets_columns_names=[c for c in df.columns if c.startswith('rides_next_')]
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40713, 674)
y_train.shape=(40713, 36)
X_test.shape=(70503, 674)
y_test.shape=(70503, 36)


In [3]:
import xgboost as xgb

In [4]:
# use only past rides data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [5]:
model = xgb.XGBRegressor()
model.fit(X_train_only_numeric, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [6]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

array([[ 1.5601035 ,  1.3401357 ,  0.02426209, ...,  1.5457414 ,
         1.9311773 ,  2.8536594 ],
       [ 3.7590663 ,  1.4488322 , -0.07925513, ...,  2.5916474 ,
         4.908646  ,  2.854332  ],
       [ 2.4798326 ,  1.2453306 ,  0.37254122, ..., -0.20751803,
         1.2950553 ,  0.9405526 ],
       ...,
       [ 0.36880848,  0.26842114,  0.21617861, ...,  0.45184943,
         0.53920954,  0.33215246],
       [ 0.7097338 ,  0.08962848,  0.2195925 , ...,  0.7319126 ,
         0.36206734,  0.5940909 ],
       [ 0.552232  ,  0.38717625,  0.6991058 , ...,  0.7885607 ,
         0.6633604 ,  0.13949385]], dtype=float32)

In [7]:
# Como da algun valor negativo como resultado eliminamos las predicciones negativas y los intervalos negativos reemplazando por cero
import numpy as np

#Limitar los valores predichos a un mínimo de cero
predictions = np.clip(predictions, 0, None)

In [8]:
predictions

array([[1.5601035 , 1.3401357 , 0.02426209, ..., 1.5457414 , 1.9311773 ,
        2.8536594 ],
       [3.7590663 , 1.4488322 , 0.        , ..., 2.5916474 , 4.908646  ,
        2.854332  ],
       [2.4798326 , 1.2453306 , 0.37254122, ..., 0.        , 1.2950553 ,
        0.9405526 ],
       ...,
       [0.36880848, 0.26842114, 0.21617861, ..., 0.45184943, 0.53920954,
        0.33215246],
       [0.7097338 , 0.08962848, 0.2195925 , ..., 0.7319126 , 0.36206734,
        0.5940909 ],
       [0.552232  , 0.38717625, 0.6991058 , ..., 0.7885607 , 0.6633604 ,
        0.13949385]], dtype=float32)

In [9]:
predictions=pd.DataFrame(predictions, columns=[f'rides_next_{i+1}_hour' for i in range(36)])
predictions

,rides_next_1_hour,rides_next_2_hour,rides_next_3_hour,rides_next_4_hour,rides_next_5_hour,rides_next_6_hour,rides_next_7_hour,rides_next_8_hour,rides_next_9_hour,rides_next_10_hour,...,rides_next_27_hour,rides_next_28_hour,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour
0,1.560104,1.340136,0.024262,0.771846,0.000000,0.095930,0.759368,2.057024,0.624711,3.638666,...,0.684104,0.144575,0.133228,0.000000,0.602612,2.939464,0.413864,1.545741,1.931177,2.853659
1,3.759066,1.448832,0.000000,0.155607,0.000000,0.536547,0.323803,1.231719,3.196930,3.845466,...,0.402183,0.288807,0.000000,0.600376,0.614647,2.163358,3.906556,2.591647,4.908646,2.854332
2,2.479833,1.245331,0.372541,0.173225,0.357720,0.311086,1.191602,1.463427,5.285834,2.488470,...,0.619779,0.538073,0.254514,0.205699,0.329713,0.000000,1.320043,0.000000,1.295055,0.940553
3,1.308443,0.778757,0.906018,0.066104,0.096851,0.226197,0.016694,0.158377,0.000000,0.512461,...,0.383959,0.000000,0.168856,0.504794,0.102982,0.000000,0.000000,0.196974,1.228848,1.223478
4,0.771065,0.890489,0.286113,0.049856,0.130031,0.148607,0.110848,0.000000,0.000000,1.014778,...,0.240178,0.139278,0.158819,0.217052,0.648669,1.619996,2.437917,3.226144,2.458884,3.095423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70498,0.449665,0.207391,0.247625,0.057638,0.073127,0.264543,0.049789,0.530443,0.318301,0.861000,...,0.083969,0.040276,0.012477,0.242000,0.102017,0.585304,0.277496,0.922149,0.591744,0.799175
70499,0.021069,0.121842,0.102315,0.030866,0.002063,0.659630,0.272618,0.517221,0.607962,0.364830,...,0.189174,0.075548,0.000000,0.434366,0.101694,0.566752,0.881637,0.436023,0.772735,1.113795
70500,0.368808,0.268421,0.216179,0.018674,0.000000,0.285931,0.000000,0.480756,0.634562,0.656835,...,0.325864,0.198605,0.089338,0.054356,0.000000,0.779945,0.517042,0.451849,0.539210,0.332152
70501,0.709734,0.089628,0.219592,0.430131,0.339959,0.086787,0.157663,0.698628,1.150640,0.642622,...,0.430465,0.043389,0.000000,0.264816,0.238831,0.493252,0.461559,0.731913,0.362067,0.594091


In [10]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=0.7034


In [11]:
#from sktime.forecasting.compose import MultioutputForecaster
#from sktime.forecasting.model_selection import temporal_train_test_split
#from sktime.forecasting.xgboost import XGBoostRegressor

import numpy as np
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#from xgboost import XGBRegressor

In [12]:
len(X_test_only_numeric)

70503

In [13]:
X_test_only_numeric.loc[1]

rides_previous_672_hour    3.0
rides_previous_671_hour    4.0
rides_previous_670_hour    1.0
rides_previous_669_hour    0.0
rides_previous_668_hour    0.0
                          ... 
rides_previous_5_hour      5.0
rides_previous_4_hour      4.0
rides_previous_3_hour      6.0
rides_previous_2_hour      4.0
rides_previous_1_hour      3.0
Name: 1, Length: 672, dtype: float32

In [14]:
X_test_only_numeric.loc[500]

rides_previous_672_hour    0.0
rides_previous_671_hour    0.0
rides_previous_670_hour    0.0
rides_previous_669_hour    0.0
rides_previous_668_hour    0.0
                          ... 
rides_previous_5_hour      0.0
rides_previous_4_hour      0.0
rides_previous_3_hour      1.0
rides_previous_2_hour      0.0
rides_previous_1_hour      0.0
Name: 500, Length: 672, dtype: float32

In [ ]:
##No voy a realizarlo de esta manera porque no arrastra errores

In [17]:
# # Realizar la predicción de múltiples pasos en los datos de prueba
# predictions_2 = []
# for i in tqdm(range(len(X_test_only_numeric))):
#     current_x = X_test_only_numeric.loc[i]
#     current_y = []
#     for j in range(36):  # especifica el número de pasos de predicción deseados
#         current_y.append(model.predict(np.array([current_x]))[0][0])
#         current_x = np.roll(current_x, -1)
#         current_x[-1] = current_y[-1]  # actualiza el último valor para la siguiente predicción
#     predictions_2.append(current_y)
# #Lo transformo en DF con los nombre de columna que corresponde
# predictions_2=pd.DataFrame(predictions_2, columns=[f'rides_next_{i+1}_hour' for i in range(36)])

In [18]:
# # Como da algun valor negativo como resultado eliminamos las predicciones negativas y los intervalos negativos reemplazando por cero
# import numpy as np

# #Limitar los valores predichos a un mínimo de cero
# predictions_2 = np.clip(predictions_2, 0, None)

In [19]:
# from sklearn.metrics import mean_absolute_error
# test_mae = mean_absolute_error(y_test, predictions_2)
# print(f'{test_mae=:.4f}')

In [20]:
# y_train.shape[1]